In [1]:
import os
import pandas as pd

from copy import copy
from openml import datasets
from openml.exceptions import OpenMLServerException
from tqdm import tqdm

In [2]:
base_path = os.getenv('OPENML_BASE')

## Data Prep

In [4]:
oml_catalog = datasets.list_datasets(output_format="dataframe")
# Remove datasets with server errors
oml_catalog = oml_catalog[~oml_catalog.did.isin([4537, 4546, 4562, 40864, 41190, 41949])]

oml_catalog.to_parquet(os.path.join(base_path, "oml_catalog.pq"), index=False)

In [5]:
ds_list = []
errors = []

for id in tqdm(oml_catalog.did):
    try:
        ds_list.append(datasets.get_dataset(id, download_qualities=True))
    except OpenMLServerException as e:
        # Error code for quality information not being available
        if e.code == 362:
            ds_list.append(datasets.get_dataset(id, download_qualities=False))
        else:
            errors.append((id, type(e), e.args))
    except Exception as e:
        errors.append((id, type(e), e.args))
    finally:
        for id, e, args in errors:
            print(f"{id}: {e}\n{args}\n")

100%|██████████| 4373/4373 [06:14<00:00, 11.69it/s]


In [6]:
md_list = []
f_list = []
q_list = []
t_list = []

for ds in ds_list:
    metadata = copy(vars(ds))

    # List or dict attributes go to separate tables for normalization
    features = metadata.pop("features", {})
    qualities = metadata.pop("qualities", {})
    tags = metadata.pop("tag", [])
    creators = metadata.pop("creator", [])
    contributors = metadata.pop("contributor", [])
    ignore_attributes = metadata.pop("ignore_attribute", [])

    if ignore_attributes:
        if len(ignore_attributes) == 1:
            ignore_attributes = ignore_attributes[0].split(",")
    else:
        ignore_attributes = []

    for k, v in features.items():
        f = copy(vars(v))
        f["dataset_id"] = ds.dataset_id
        f["ignore"] = False
        if f["name"] in ignore_attributes:
            f["ignore"] = True
        f_list.append(f)

    if qualities is not None:
        for k, v in qualities.items():
            q_list.append({"dataset_id": ds.dataset_id, "metric": k, "value": v})

    if tags is not None:
        for t in tags:
            t_list.append({"dataset_id": ds.dataset_id, "tag": t})

    # The following attributes do not contain any data
    _ = metadata.pop("update_comment", None)
    _ = metadata.pop("_dataset", None)
    _ = metadata.pop("data_pickle_file", None)
    _ = metadata.pop("data_feather_file", None)
    _ = metadata.pop("feather_attribute_file", None)

    # The following attributes always contain the same data
    _ = metadata.pop("cache_format", None)
    _ = metadata.pop("format", None)
    _ = metadata.pop("visibility", None)

    md_list.append(metadata)

dataset_df = pd.DataFrame(md_list)
feature_df = pd.DataFrame(f_list)
quality_df = pd.DataFrame(q_list)
tags_df = pd.DataFrame(t_list)

col = feature_df.pop("dataset_id")
feature_df.insert(0, col.name, col)

In [7]:
dataset_df.to_parquet(os.path.join(base_path, "datasets.pq"), index=False)
feature_df.to_parquet(os.path.join(base_path, "features.pq"), index=False)
quality_df.to_parquet(os.path.join(base_path, "metrics.pq"), index=False)
tags_df.to_parquet(os.path.join(base_path, "tags.pq"), index=False)


## Analysis

In [8]:
oml_catalog.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4373 entries, 2 to 44256
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   did                                 4373 non-null   int64  
 1   name                                4373 non-null   object 
 2   version                             4373 non-null   int64  
 3   uploader                            4373 non-null   object 
 4   status                              4373 non-null   object 
 5   format                              4373 non-null   object 
 6   MajorityClassSize                   1563 non-null   float64
 7   MaxNominalAttDistinctValues         1401 non-null   float64
 8   MinorityClassSize                   1563 non-null   float64
 9   NumberOfClasses                     3346 non-null   float64
 10  NumberOfFeatures                    4334 non-null   float64
 11  NumberOfInstances                   4334 n

In [9]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4373 entries, 0 to 4372
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   dataset_id                4373 non-null   int64 
 1   name                      4373 non-null   object
 2   version                   4373 non-null   int64 
 3   description               4109 non-null   object
 4   collection_date           2583 non-null   object
 5   upload_date               4373 non-null   object
 6   language                  1361 non-null   object
 7   licence                   4373 non-null   object
 8   url                       4373 non-null   object
 9   default_target_attribute  3493 non-null   object
 10  row_id_attribute          1186 non-null   object
 11  version_label             1191 non-null   object
 12  citation                  676 non-null    object
 13  original_data_url         1751 non-null   object
 14  paper_url               

In [10]:
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4158111 entries, 0 to 4158110
Data columns (total 7 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   dataset_id             int64 
 1   index                  int64 
 2   name                   object
 3   data_type              object
 4   nominal_values         object
 5   number_missing_values  int64 
 6   ignore                 bool  
dtypes: bool(1), int64(3), object(3)
memory usage: 194.3+ MB


In [11]:
features_cleansed = feature_df[ \
    ~feature_df.name.str.contains("^oz[1-9][0-9]?[0-9]?$") & \
    ~feature_df.name.str.contains("^V[0-9][0-9]?[0-9]?$") & \
    ~feature_df.name.str.contains("^col_[0-9][0-9]?[0-9]?$") & \
    ~feature_df.name.str.contains("AFFX-") & \
    ~feature_df.name.str.contains("Var\d{0,7}") & \
    ~feature_df.name.str.contains("att_\d{0,7}") & \
    ~feature_df.name.str.contains("^\d+$") & \
    ~feature_df.name.str.contains("\d{2,7}(?:_\w*)?_at$")
].merge(dataset_df[~dataset_df.name.str.startswith("QSAR-TID")].dataset_id, how="right", on="dataset_id")

In [12]:
feature_count = features_cleansed.groupby("name")["dataset_id"].count()

In [13]:
feature_count.sort_values(ascending=False).to_csv(os.path.join(base_path, "feature_count.csv"), header=["count"])

In [14]:
features_cleansed

,dataset_id,index,name,data_type,nominal_values,number_missing_values,ignore
0,2,0.0,family,nominal,"[GB, GK, GS, TN, ZA, ZF, ZH, ZM, ZS]",772.0,False
1,2,1.0,product-type,nominal,"[C, G, H]",0.0,False
2,2,2.0,steel,nominal,"[A, K, M, R, S, U, V, W]",86.0,False
3,2,3.0,carbon,numeric,None,0.0,False
4,2,4.0,hardness,numeric,None,0.0,False
...,...,...,...,...,...,...,...
1324711,44255,1.0,CATEGORY,string,None,0.0,False
1324712,44255,2.0,SUPER_CATEGORY,numeric,None,800.0,False
1324713,44256,0.0,FILE_NAME,string,None,0.0,False
1324714,44256,1.0,CATEGORY,string,None,0.0,False
